<a href="https://colab.research.google.com/github/Cliyac/LLM_Project/blob/main/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate numpy tqdm bertscore torch

ERROR: Could not find a version that satisfies the requirement bertscore (from versions: none)
ERROR: No matching distribution found for bertscore


In [ ]:
!pip install -q datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
!pip install bert-score torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
!pip install evaluate

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import evaluate
import numpy as np
from tqdm import tqdm
import json
from typing import List, Dict, Any


In [ ]:


def load_model_and_tokenizer(model_path: str):
    """Load the fine-tuned model and tokenizer from Hugging Face."""
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

def generate_response(
    model,
    tokenizer,
    prompt: str,
    max_length: int = 512,
    temperature: float = 0.7,
    top_p: float = 0.9,
    num_return_sequences: int = 1
) -> str:
    """Generate a response from the model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def evaluate_rouge(predictions: List[str], references: List[str]) -> Dict[str, float]:
    """Calculate ROUGE scores."""
    rouge = evaluate.load("rouge")
    results = rouge.compute(predictions=predictions, references=references)
    return results

def evaluate_bleu(predictions: List[str], references: List[str]) -> float:
    """Calculate BLEU score."""
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=predictions, references=references)
    return results["bleu"]

def evaluate_bertscore(predictions: List[str], references: List[str]) -> Dict[str, float]:
    """Calculate BERTScore."""
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions, references=references, lang="en")
    return {
        "bertscore_precision": np.mean(results["precision"]),
        "bertscore_recall": np.mean(results["recall"]),
        "bertscore_f1": np.mean(results["f1"])
    }

# Load model and tokenizer
model_path = "cliyac/Llama-2-7b-student-assistant"  # Replace with your model ID
model, tokenizer = load_model_and_tokenizer(model_path)
print(f"Model loaded successfully: {model_path}")

# Create test data
test_data = [
    {
        "input": "What is the capital of France?",
        "output": "The capital of France is Paris."
    },
    {
        "input": "Explain the concept of machine learning in one sentence.",
        "output": "Machine learning is a field of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed."
    },
    {
        "input": "Write a short poem about technology.",
        "output": "In circuits and code we find our way,\nThrough digital dreams we learn to play,\nInnovation guides us day by day,\nAs technology shapes our modern way."
    },
    {
        "input": "What are the three primary colors?",
        "output": "The three primary colors are red, blue, and yellow."
    },
    {
        "input": "Summarize the benefits of exercise in one sentence.",
        "output": "Regular exercise improves physical health, mental well-being, and overall quality of life."
    }
]

print(f"Created {len(test_data)} test samples")

# Set maximum number of samples to evaluate
max_samples = 100
if max_samples:
    test_data = test_data[:min(max_samples, len(test_data))]

predictions = []
references = []

# Generate predictions
for item in tqdm(test_data, desc="Generating predictions"):
    prompt = item["input"]
    reference = item["output"]

    prediction = generate_response(model, tokenizer, prompt)
    predictions.append(prediction)
    references.append(reference)

    # Print example prediction (first few samples)
    if len(predictions) <= 3:
        print(f"\nExample {len(predictions)}:")
        print(f"Prompt: {prompt}")
        print(f"Prediction: {prediction}")
        print(f"Reference: {reference}")

# Calculate metrics
metrics = {
    "rouge": evaluate_rouge(predictions, references),
    "bleu": evaluate_bleu(predictions, references),
    "bertscore": evaluate_bertscore(predictions, references)
}

# Print results
print("\nEvaluation Results:")
print(f"ROUGE Scores: {metrics['rouge']}")
print(f"BLEU Score: {metrics['bleu']}")
print(f"BERTScore: {metrics['bertscore']}")

# Save results to file
with open("evaluation_results.json", "w") as f:
    json.dump(metrics, f, indent=4)
print("\nResults saved to evaluation_results.json")

# Download results
from google.colab import files
files.download('evaluation_results.json')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Model loaded successfully: cliyac/Llama-2-7b-student-assistant
Created 5 test samples


Generating predictions:   0%|          | 0/5 [00:00<?, ?it/s]